# GUI Based Prediction Notebook

In [2]:
#import the required library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [13]:
#load the apple dataset
apple = pd.read_csv('D:/AI/apple_prediction/AAPL.csv')

In [14]:
# load the test data
test_data = pd.read_csv('D:/AI/apple_prediction/AAPL_test.csv', parse_dates=['Date'])

In [15]:
# Load the trained LSTM model
model = load_model('D:/AI/apple_prediction/best_model.h5')

In [16]:
#chose Close for y
apple["Close"] = pd.to_numeric(apple.Close, errors = 'coerce' )
apple = apple.dropna()
#create a numpy arrary not a vector:
train_data = apple.iloc[:,4:5].values

In [17]:
#scale the test data between 0 to 1
sc = MinMaxScaler(feature_range = (0,1))
train_data = sc.fit_transform(train_data)
train_data.shape

(5036, 1)

## GUI-1: Display the predicted price of the next day

In [31]:
# Create a tkinter window for next day price prediction
root = tk.Tk()
root.geometry("300x100")
root.title("Stock Price Predictor")
# Define a function to update the predicted price label
def update_predicted_price():
    # Call the function to get the predicted price
    predicted_price = get_predicted_price()
    # Update the label with the predicted price
    predicted_price_label.config(text=f"Predicted price: ${predicted_price:.2f}")
    
# Define a function to get the predicted price
def get_predicted_price():
    apple_new = apple.filter(['Close'])
    #last 2 months (60 days) closing prices are
    last_60_days = apple_new[-60:].values
    last_60_days_scaled = sc.transform(last_60_days)
    X_test = []
    X_test.append(last_60_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))
    next_day_pred = model.predict(X_test)
    next_day_pred  = sc.inverse_transform(next_day_pred)
    return next_day_pred[0][0]

# Create a Label widget to display the predicted price
predicted_price_label = tk.Label(root, text="Predicted price: $0.00")

# Create a button to update the predicted price
update_button = tk.Button(root, text="Update", command=update_predicted_price)

# Pack the widgets into the window
predicted_price_label.pack()
update_button.pack()

# Run the tkinter event loop
root.mainloop()

1/1 [==============================] - 0s 34ms/step


In [21]:
#covert the date column into date time using pandas
apple['Date'] = pd.to_datetime(apple['Date'])
# extract the closing price for the specified date
apple_new = apple.filter(['Close'])
apple_actual = apple.filter(['Close'])
next_day_act = apple_new[train_data.shape[0]-1:train_data.shape[0]].values

In [23]:
apple_new = apple.filter(['Close'])
#last 2 months (60 days) closing prices are
last_60_days = apple_new[-60:].values
last_60_days_scaled = sc.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))
next_day_pred = model.predict(X_test)
next_day_pred  = sc.inverse_transform(next_day_pred)

1/1 [==============================] - 0s 39ms/step


In [33]:
# Create a tkinter window for suggestion
root = tk.Tk()
root.geometry("300x250")
root.title("Stock Price Predictor-2")

# Define a function to update the predicted price label
def update_predicted_price():
    # Call the function to get the predicted price
    predicted_price = get_predicted_price()
    # Update the label with the predicted price
    predicted_price_label.config(text=f"Predicted price: ${predicted_price:.2f}")

# Define a function to update 
def update_stop_loss():
    # Call the function to get the predicted price
    stop_loss = get_stop_loss()
    # Update the label with the predicted price
    stop_loss_label.config(text=f"Suggested stop loss level: ${stop_loss:.2f}")

# Define a function to update 
def update_take_profit():
    # Call the function to get the predicted price
    take_profit = get_take_profit()
    # Update the label with the predicted price
    take_profit_label.config(text=f"Suggested take profit level: ${take_profit:.2f}")    

# Define a function to get the predicted price
def get_predicted_price():
    apple_new = apple.filter(['Close'])
    #last 2 months (60 days) closing prices are
    last_60_days = apple_new[-60:].values
    last_60_days_scaled = sc.transform(last_60_days)
    X_test = []
    X_test.append(last_60_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))
    next_day_pred = model.predict(X_test)
    next_day_pred  = sc.inverse_transform(next_day_pred)
    return next_day_pred[0][0]
def get_stop_loss():
    # set the stop loss and take profit thresholds as percentages of the actual price
    stop_loss_threshold = 0.02 # 2% of actual price
    stop_loss_level = next_day_act - (stop_loss_threshold * next_day_pred)
    return stop_loss_level[0][0]
def get_take_profit():
    take_profit_threshold = 0.03 # 3% of actual price
    take_profit_level = next_day_act + (take_profit_threshold * next_day_pred)
    return take_profit_level[0][0]

# Create a Label widget to display the predicted price
predicted_price_label = tk.Label(root, text="Predicted price: $0.00")
# Create a button to update the predicted price
update_button = tk.Button(root, text="Update", command=update_predicted_price)
# Pack the widgets into the window
predicted_price_label.pack()
update_button.pack()

# Create a Label widget to display the 
stop_loss_label = tk.Label(root, text="Update please")
# Create a button to update the 
update_button_st = tk.Button(root, text="Update", command=update_stop_loss)
# Pack the widgets into the window
stop_loss_label.pack()
update_button_st.pack()

# Create a Label widget to display the 
take_profit_label = tk.Label(root, text="Update please")
# Create a button to update the 
update_button_tp = tk.Button(root, text="Update", command=update_take_profit)
# Pack the widgets into the window
take_profit_label.pack()
update_button_tp.pack()

# Run the tkinter event loop
root.mainloop()

1/1 [==============================] - 0s 34ms/step


## GUI 2: using yyyy-mm-dd

In [34]:
import tkinter as tk
from tkinter import ttk
import numpy as np
from datetime import datetime, timedelta

# create the Tkinter window
root = tk.Tk()
root.title("Stock Price Predictor")

# define a function to update the predicted price label
def update_predicted_price():
    # get the current value of the input field
    input_value = input_field.get()

    # convert the input value to a datetime object
    input_date = datetime.strptime(input_value, '%Y-%m-%d')

    # create a 3-dimensional array with a single sample
    # for the LSTM model to predict
    X_pred = np.zeros((1, n_steps, n_features))
    for i in range(n_steps):
        date = input_date - timedelta(days=n_steps-i)
        if date.date() in test_data.index:
            X_pred[0, i, :] = test_data.loc[date.date(), :].values
        else:
            # if the date is not in the data, use the last available data point
            X_pred[0, i, :] = X_test[-1, i, :]

    # perform the prediction using the LSTM model and get the predicted price
    predicted_price = model.predict(X_pred)[0, 0]

    # update the label with the predicted price
    predicted_price_label.config(text=f"Predicted price: ${predicted_price:.2f}")

# define the number of time steps and features
n_steps = 30
n_features = 1

# create a label for the input field
input_label = ttk.Label(root, text="Enter a date (YYYY-MM-DD):")
input_label.pack()

# create an input field
input_field = ttk.Entry(root)
input_field.pack()

# create a button to trigger the prediction
predict_button = ttk.Button(root, text="Predict", command=update_predicted_price)
predict_button.pack()

# create a label for the predicted price
predicted_price_label = ttk.Label(root, text="Predicted price: $0.00")
predicted_price_label.pack()

# start the Tkinter event loop
root.mainloop()


1/1 [==============================] - 0s 31ms/step
